In [ ]:
import os
import json
import pydantic

import langchain_openai

# docs: https://python.langchain.com/docs/how_to/structured_output/

credentials = {
    "api_key": os.environ["AZURE_OPENAI_API_KEY"],
    "azure_endpoint": os.environ["AZURE_OPENAI_ENDPOINT"],
    "api_version": "2025-01-01-preview",
}

# Model & Client
model = "gpt-4o-mini"
client = langchain_openai.AzureChatOpenAI(azure_deployment=model, **credentials)


___
### From pydantic.BaseModel

In [ ]:
class Schema(pydantic.BaseModel):
    setup: str = pydantic.Field(description="The setup of the joke")
    punchline: str = pydantic.Field(description="The punchline to the joke")
    rating: int = pydantic.Field(description="How funny the joke is, from 1 to 10")


client_with_structured_output = client.with_structured_output(Schema)
response = client_with_structured_output.invoke("Tell me a joke about cats")
print(f"setup: {response.setup}")
print(f"punchline: {response.punchline}")
print(f"rating: {response.rating}")


___
### From dict

In [ ]:
schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

client_with_structured_output = client.with_structured_output(schema)
response = client_with_structured_output.invoke("Tell me a joke about cats")
print(json.dumps(response, indent=2))


In [ ]:
# NOTE: To make a property non-generative: set default value, remove its 'description' and add it to required
schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            # 'description': 'How funny the joke is, from 1 to 10',
            "default": 5,
        },
    },
    "required": ["setup", "punchline", "rating"],
}

client_with_structured_output = client.with_structured_output(schema)
response = client_with_structured_output.invoke("Tell me a joke about cats")
print(json.dumps(response, indent=2))
